# IBM Data Science Capstone Project :: Week 4 & 5

**Analyzing Lahore City to find best town w.r.t nearby facilities**

Lahore is a city of Pakistan. There are few town that are old and few are built in recent past. People come here to seek jobs and to start their own business. For a person planning to settle in Lahore, he/ she will be having difficulty to choose the best town. More facilites surronded by the town may be a perfect choice for someone. This project aims to provide a overview of all the town w.r.t nearby venues like resturants, shopping malls etc.

**Data Used in Project**

*Data Prepration Phase* : Data of Towns in Lahore is available in Excel Format.

Geographical coordinates of the Town will be getting from online sources (OpenStreetMap or arcgis)

Obtain the venue data for the Towns from **Foursquare API**

Explore and cluster the Towns

Select the best Town